# 🏗️ Step 1: Download REIT Stock Data

In [ ]:
import yfinance as yf
import pandas as pd
import os

# 设置股票代码（房地产相关）
tickers = ['PLD', 'SPG', 'O']  # Prologis, Simon Property, Realty Income
start_date = '2020-01-01'
end_date = '2024-01-01'
data_path = 'data/'

# 下载股票数据并保存为 CSV 文件
for ticker in tickers:
    stock = yf.download(ticker, start=start_date, end=end_date)
    stock.to_csv(os.path.join(data_path, f"{ticker}.csv"))
    print(f"{ticker} data saved.")
